https://colab.research.google.com/drive/1YAYRUe2VqV22po1fhVHvoWZjkHFuyAjk?usp=sharing

# Setup

In [ ]:
!pip install -q streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 36.7 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@

In [ ]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1-CKKgpis2PShA3wQL1F9Rkq5i-Yv6hCm?usp=drive_link", quiet=True)
gdown.download_folder("https://drive.google.com/drive/folders/1QiTPba9c916OZLXHU9c3F_VN_TQ-PEYm?usp=sharing", quiet=True)
gdown.download_folder("https://drive.google.com/drive/folders/1Eq9vKx2fNiG1zSA6CHRS4cSNJrvu5soI?usp=sharing",quiet=True)

['/content/model_ALexNet/accuracy_test.png',
 '/content/model_ALexNet/accuracy_val.png',
 '/content/model_ALexNet/AlexNet.png',
 '/content/model_ALexNet/Capture.PNG',
 '/content/model_ALexNet/fingerprint.pb',
 '/content/model_ALexNet/keras_metadata.pb',
 '/content/model_ALexNet/loss_test.png',
 '/content/model_ALexNet/loss.png',
 '/content/model_ALexNet/model.png',
 '/content/model_ALexNet/predict.png',
 '/content/model_ALexNet/saved_model.pb',
 '/content/model_ALexNet/training.png',
 '/content/model_ALexNet/variables/variables (1).data-00000-of-00001',
 '/content/model_ALexNet/variables/variables.data-00000-of-00001',
 '/content/model_ALexNet/variables/variables.index',
 '/content/model_ALexNet/variables (1).data-00000-of-00001',
 '/content/model_ALexNet/variables.index']

#Create python script


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import time
import os
import math
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split

#######cache#####

#######VOCAB#####
characters=['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
characters = sorted(list(set(characters)))
# Mapping characters to integers
char_to_num = layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

#######LOAD MODEL############
#@st.cache_resource
def initial(model):
  image = cv2.imread('/content/model_ALexNet/Capture.PNG')
  tensor=image[:, :, :3]/255
  tensor= tf.image.transpose(tensor)
  tensor = tf.image.resize(tensor, [200, 50])
  image =tf.image.resize(tensor, [200, 50])
  _image= tf.reshape(image, (1, 200, 50, 3))
  model.predict(_image, verbose=0)

@st.cache_resource
def load_model_CNN():
  model=tf.keras.saving.load_model("/content/CNN/ocr_model")
  initial(model)
  return model
@st.cache_resource
def load_model_Res():
  model= tf.keras.saving.load_model("/content/resnet50v2.keras")
  initial(model)
  return model
@st.cache_resource
def load_model_Alex():
  model= tf.keras.saving.load_model("/content/model_ALexNet")
  initial(model)
  return model
CNN_model_pred=load_model_CNN()
RES_model_pred=load_model_Res()
ALEX_mode_pred=load_model_Alex()
@st.cache_resource
def getPredictor(choice):
  if choice==0:
    return CNN_model_pred
  elif choice==1:
    return RES_model_pred
  elif choice==2:
    return ALEX_mode_pred
#######PREDICT############
def predictingggg(image,choice):
  start_time = time.time()
  tensor= tf.image.decode_image(image)/255
  tensor=tensor[:, :, :3]
  tensor= tf.image.transpose(tensor)
  tensor = tf.image.resize(tensor, [200, 50])
  image =tf.image.resize(tensor, [200, 50])
  _image= tf.reshape(image, (1, 200, 50, 3))
  #st.write("Runtime: {:.2f} seconds".format(start_time-time.time()))
  start_time = time.time()
  pred = getPredictor(choice).predict(_image, verbose=0)
  #st.write("Runtime: {:.2f} seconds".format(start_time-time.time()))
  start_time = time.time()
  pred_text = decode_batch_predictions(pred)
  return pred_text[0]

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :5
    ]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text
def main():
    choices = ['CNN', 'ResNet50',"AlexNet"]
    selected_option = st.selectbox('Select a model', choices)

    # File uploader
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        # Get the name of the uploaded file
        image_name = uploaded_file.name

        # Read the uploaded image file
        image = uploaded_file.read()
        selected_index = choices.index(selected_option)
        if(image!=None):
          result=predictingggg(image,selected_index)
          st.image(uploaded_file, caption=image_name, use_column_width=True)
          st.write("result:", result)

if __name__ == "__main__":
    main()



Writing app.py


# Run streamlit in background

In [ ]:
import urllib
print("Password: ",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password:  35.204.86.231
npx: installed 22 in 2.065s
your url is: https://bright-cloths-lose.loca.lt
^C
